# Classificação de palavras com LSTM

Na linguagem natural, classificar uma palavra como artigo, verbo ou nome é uma tarefa extremamente simples. Há casos em que uma palavra pode ter dois significados diferentes em uma frase, dependendo do contexto. Para nós, é extremamente simples idetificar o contexto no qual uma palavra foi aplicada, uma vez que basta analisarmos o restante da frase. Para um computador, essa tarefa não é tão simples. Como o significado de uma palavra depende da frase na qual ela foi utilizada, a utilização de RNN torna-se uma boa escolha para fazer a análise da sequência (nesse caso, a frase). Neste exemplo, originalmente feito por https://github.com/LeanManager/NLP-PyTorch/blob/master/LSTM%20Speech%20Tagging%20with%20PyTorch.ipynb, utilizaremos a arquitetura LSTM para classificação de palavras em artigo, nome ou verbo. Será utilizada uma base de dados extremamente simples, pois o importante é apenas entender o funcionamento da rede LSTM.

## Dados para treinamento

Redes neurais não costumam se dar bem com palavras como dados de entrada. Portanto, em primeiro lugar iremos transformar sequências de palavras, ou frases, em vetores de números. Como utilizaremos o pytorch, faremos o uso de tensores para realizar tal tarefa.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Define o dispositivo que será utilizado (CPU ou GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

As frases que utilizaremos são extremamente simples. Primeiro, as strings serão separadas em um vetor de palavras. A seguir, será criado um dicionário para armazenar os índices correspondentes de cada palavra na nossa base de dados. O mesmo será feito com as tags. No presente exemplo, trataremos artigos, nomes e verbos, como no código abaixo.

In [2]:
# setenças para treinamento e as classificações de suas palavras
# ART = artigo
# NN = nome
# V = verbo
training_data = [
	("O gato comeu o queijo".lower().split(), ["ART", "NN", "V", "ART", "NN"]),
    ("Ela leu aquele livro".lower().split(), ["NN", "V", "ART", "NN"]),
    ("O cachorro ama arte".lower().split(), ["ART", "NN", "V", "NN"]),
    ("O elefante atende o telefone".lower().split(), ["ART", "NN", "V", "ART", "NN"])
]

# cria um dicionário para representação numérica das palavras
# palavras -> índices
word2idx = {}

for sent, tags in training_data:
    for word in sent:
        if word not in word2idx:
            word2idx[word] = len(word2idx)

print(word2idx)

# cria um dicionário para representação numérica da classificação das palavras
tag2idx = {"ART": 0, "NN": 1, "V": 2}

print(tag2idx)

{'o': 0, 'gato': 1, 'comeu': 2, 'queijo': 3, 'ela': 4, 'leu': 5, 'aquele': 6, 'livro': 7, 'cachorro': 8, 'ama': 9, 'arte': 10, 'elefante': 11, 'atende': 12, 'telefone': 13}
{'ART': 0, 'NN': 1, 'V': 2}


Para trabalhar com o pytorch, precisamos utilizar tensores. Para isso, criaremos uma função que recebe a frase e o dicionário de índices. A seguir, a função cria um vetor de índices, transforma em um numpy array e por fim em um tensor.

In [3]:
# Função que converte uma sequência de palavras em um tensor de valores numéricos
# Será usada para treinamento
def prepare_sequence(seq, to_idx):
    """
    Esta função recebe uma sequência de palavras e retorna um tensor 
    correspondente de valores numéricos (índices de cada palavra).
    """
    
    idxs = [to_idx[w] for w in seq]
    idxs = np.array(idxs)
    
    tensor = torch.from_numpy(idxs).long()

    tensor = tensor.to(device)

    return tensor

# verifica o que a função prepare_sequence faz com uma das sentenças que serão 
# utilizadas no treinamento
example_input = prepare_sequence("O elefante atende o telefone".lower().split(), word2idx)

print(example_input)

tensor([ 0, 11, 12,  0, 13], device='cuda:0')


## Criando o modelo

Nosso modelo assume que as seguintes condições são satisfeitas:

1. A entrada é uma sequência de palavras do tipo [w1, w2, ..., wn]
2. As palavras na entrada vêm de uma frase (ou uma string)
3. Temos um número de tags limitado: ART, NN, V (Artigo, Nome, Verbo
4. Queremos prever uma tag para cada palavra na entrada

Para realizar a previsão (ou predição, considerando que a saída será mostrada ou "dita" pela rede), utilizaremos uma LSTM para uma sequência de testes e aplicaremos a função softmax ao hidden state. O resultato deverá ser um tensor com a pontuação das tags a partir do qual poderemos predizer a tag de uma palavra baseado no máximo valor apresentado nas pontuações obtidas.

Matematicamente, podemos representar uma previsão de tag da seguinte maneira:

$$\hat{y}_i = \text{argmax}_j (\text{logSoftmax}(Ah_i + b))_j$$

Onde $A$ é um peso aprendido pelo modelo, $b$ um termo de bias aprendido e $h_i$ o hidden state no tempo $i$.

### Embeddings

A rede LSTM recebe como argumento um tamanho de entrada e um tamanho para o hidden state. Porém, raramente as sentenças possuem tamanho fixo. Para contornar isso, pode ser utilizada uma camada de associação logo após a entrada. Essa camada é capaz de receber uma palavra e gerar um tensor numérico de tamanho definido que a representa. Assim, ainda que a palavra varie de tamanho, após passar pela camada de associação, ela terá o tamanho definido previamente pelo usuário. 

Após passar pela camada de associação, os tensores são tratados pela camada LSTM e por fim por uma camada linear. Essa basicamente faz uma transformação linear na saída do hidden state da LSTM para que ela saia com as dimensões especificadas pelo usuário. Nesse caso, a saída para cada palavra deverá ser um vetor de 3 posições, cada uma contendo uma pontuação para cada tag: [ART, NN, V].

![Arquitetura do modelo](Imagens/Arquitetura.png)

In [4]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        ''' Initialize the layers of this model.'''
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim

        # embedding layer that turns words into a vector of a specified size
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim).to(device)

        # the LSTM takes embedded word vectors (of a specified size) as inputs 
        # and outputs hidden states of size hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim).to(device)

        # the linear layer that maps the hidden state output dimension 
        # to the number of tags we want as output, tagset_size (in this case this is 3 tags)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size).to(device)
        
        # initialize the hidden state (see code below)
        self.hidden = self.init_hidden()

        
    def init_hidden(self):
        ''' At the start of training, we need to initialize a hidden state;
           there will be none because the hidden state is formed based on perviously seen data.
           So, this function defines a hidden state with all zeroes and of a specified size.'''
        # The axes dimensions are (n_layers, batch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim).to(device),
                torch.zeros(1, 1, self.hidden_dim).to(device))

    def forward(self, sentence):
        ''' Define the feedforward behavior of the model.'''
        # create embedded word vectors for each word in a sentence
        embeds = self.word_embeddings(sentence)
        
        # get the output and hidden state by passing the lstm over our word embeddings
        # the lstm takes in our embeddings and hiddent state
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        
        # get the scores for the most likely tag for a word
        tag_outputs = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_outputs, dim=1)
        
        return tag_scores

Para este exemplo, por ser muito simples, podemos utilizar dimensões reduzidas para a camada de associação e para o hidden state da LSTM. O tamanho do vocabulário nada mais é do que o tamanho do dicionário de índices definido anteriormente. A partir dele a camada de associação será capaz de construir o tensor que representa a palavra.

In [5]:
# O dimensão da camada de embedding define o tamanho do vetor que representa cada palavra.
# Por se tratar de um exemplo extremamente simples, serão consideradas dimensões pequenas. 
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# instantiate our model
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word2idx), len(tag2idx))

Para verificar como o modelo se comporta, realizaremos um teste. Definiremos uma sentença com palavras existentes em nossa base de dados anteriormente definida. A seguir, será obtida a sequência de índices e por fim o modelo será utilizado para verificar a pontuação de cada palavra para cada tag.

In [6]:
test_sentence = "O queijo ama o elefante".lower().split()

inputs = prepare_sequence(test_sentence, word2idx)
tag_scores = model(inputs)
print(tag_scores)

# Obtém as tags com maior pontuação
_, predicted_tags = torch.max(tag_scores, 1)
print('\n')
print('Predicted tags: \n',predicted_tags)

tensor([[-0.9782, -1.3721, -0.9930],
        [-0.9247, -1.4035, -1.0283],
        [-0.9336, -1.3931, -1.0257],
        [-0.9334, -1.4338, -0.9986],
        [-1.0253, -1.2551, -1.0321]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)


Predicted tags: 
 tensor([0, 0, 0, 0, 0], device='cuda:0')


O modelo parece estar se comportanto exatamente como queremos. Agora, precisa ser treinado. Para isso, definiremos a função de perda Negative Log Likelihood (NLL) e a função de otimização Stochastic Gradient Descent (SGD). Por se tratar de um modelo simples, o número de épocas pode ser pequeno. 

In [7]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

n_epochs = 300

for epoch in range(n_epochs):
    
    epoch_loss = 0.0
    
    # get all sentences and corresponding tags in the training data
    for sentence, tags in training_data:
        
        # zero the gradients
        model.zero_grad()

        # zero the hidden state of the LSTM, this ARTaches it from its history
        model.hidden = model.init_hidden()

        # prepare the inputs for processing by out network, 
        # turn all sentences and targets into Tensors of numerical indices
        sentence_in = prepare_sequence(sentence, word2idx)
        targets = prepare_sequence(tags, tag2idx)

        # forward pass to get tag scores
        tag_scores = model(sentence_in)

        # compute the loss, and gradients 
        loss = loss_function(tag_scores, targets)
        epoch_loss += loss.item()
        loss.backward()
        
        # update the model parameters with optimizer.step()
        optimizer.step()
        
    # print out avg loss per 20 epochs
    if(epoch%20 == 19):
        print("Epoch: %d, loss: %1.5f" % (epoch+1, epoch_loss/len(training_data)))

Epoch: 20, loss: 0.94513
Epoch: 40, loss: 0.72202
Epoch: 60, loss: 0.46925
Epoch: 80, loss: 0.28682
Epoch: 100, loss: 0.17411
Epoch: 120, loss: 0.11429
Epoch: 140, loss: 0.08104
Epoch: 160, loss: 0.06094
Epoch: 180, loss: 0.04782
Epoch: 200, loss: 0.03870
Epoch: 220, loss: 0.03205
Epoch: 240, loss: 0.02704
Epoch: 260, loss: 0.02315
Epoch: 280, loss: 0.02009
Epoch: 300, loss: 0.01764


Com o modelo treinado, realizaremos um teste. Por se tratar de um exemplo extremamente simples, espera-se que o modelo se saia bem. A frase escolhida foi "O queijo ama o elefante". Observe que 'O' é um artigo, 'queijo' um nome, 'ama' um verbo, 'o' um artigo e 'elefante' um nome. Assim, a classificação correta é: [0, 1, 2, 0, 1]

In [8]:
test_sentence = "O queijo ama o elefante".lower().split()

# Verifica a pontuação após o treinamento
inputs = prepare_sequence(test_sentence, word2idx)
tag_scores = model(inputs)
print(tag_scores)

# Obtém a pontuação máxima para cada tag
_, predicted_tags = torch.max(tag_scores, 1)
print('\n')
print('Predicted tags: \n',predicted_tags)

tensor([[-4.6825e-03, -6.0592e+00, -6.0596e+00],
        [-6.5345e+00, -1.0237e-02, -4.7407e+00],
        [-4.8942e+00, -4.1038e+00, -2.4293e-02],
        [-1.3391e-02, -6.0029e+00, -4.5254e+00],
        [-6.3960e+00, -1.4984e-02, -4.3272e+00]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)


Predicted tags: 
 tensor([0, 1, 2, 0, 1], device='cuda:0')


## Finalizando

O modelo se comportou bem e conseguiu acertar as tags de cada palavra. Foi um exemplo extremamente simples, mas que mostrou o funcionamento da LSTM. Para exemplos maiores e mais complexos, uma base de dados maior poderia ser utilizada. Assim, vimos que uma rede LSTM pode ser utilizada para classificar palavras como artigo, verbo e nome, mas não para por aí. Redes que utilizam arquitetura semelhante podem por exemplo traduzir textos! O google tradutor funciona dessa maneira. 